In [ ]:
from movingpeople import visualise_route, generate_routes

from shapely import Point
import geopandas as gpd

In [ ]:
help(generate_routes)

In [ ]:
help(visualise_route)

In [ ]:
import osmnx as ox

# Search query for a geographic area
query = "City of Westminster"
# Get the walking network for the query location
G = ox.graph.graph_from_place(query, network_type="walk", simplify=True)
# Project the graph to WGS84
Gp = ox.project_graph(G, to_crs="4326")

In [ ]:
# To make randomised routes from a single origin
routes = generate_routes(
                            Gp,
                            time_from="2020-02-26 20:42:53",
                            time_until="2020-02-26 21:42:53",
                            time_strategy="random",
                            route_strategy="one-many",
                            origin_destination_coords=[51.499127, -0.153522],
                            total_routes=3,
                            walk_speed=1.4,
                            frequency="1s",
                            )

In [ ]:
routes

In [ ]:
# Creating a Point geometry and a buffer polygon of 500 meters
buffer = Point(-0.15258859375821948, 51.5120199884501792)
area = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[buffer])
area = area.to_crs(crs=3857) 
area['geometry'] = area['geometry'].buffer(500)

In [ ]:
def clip_routes_to_polygon(routes, polygon):
    """
    Creates a DataFrame of routes that are clipped by a single polygon.

    Parameters:
            routes : GeoDataFrame
                DataFrame containing routes locations. See 'generate_routes' for information.
            polygon : GeoDataFrame
                Contains polygon 'geometry' column

    Returns:
            route_subset : GeoDataFrame
                A subset of routes that are clipped inside the input polygon.
    """
    
    # Check if polygon is a Shapely Polygon
    assert polygon.geom_type == 'Polygon', "Input polygon is not a Shapely 'Polygon'."

    # Check if the geometry column exists in the polygon GeoDataFrame
    assert 'geometry' in area.columns, "The polygon GeoDataFrame doesnt have a column named 'geometry'."

    # Check if the geometry column exists in the routes GeoDataFrame
    assert 'geometry' in routes.columns, "The polygon GeoDataFrame doesnt have a column named 'geometry'."

    # Check if the geometries in the routes GeoDataFrame
    assert routes.geom_type == 'Point', "Input polygon is not a Shapely 'Polygon'."

    # Subsetting routes to within the buffer polygon - FUNCTION
    route_subset = routes.loc[routes.within(polygon.loc[0, 'geometry'])]
    return route_subset

In [ ]:
# making a kepler GL map
map_routes = visualise_route(routes, 600)

# Adding the buffer geometry
map_routes.add_data(data=area, name='buffer')

# Adding the subset routes to the map
map_routes.add_data(data=route_subset, name='route_subset')

# Display the map
map_routes

In [ ]:
def get_entry_exit_times(clipped_routes):

    """
    Gets the start and end times of unique routes.

    Parameters:
            clipped_routes : GeoDataFrame
                DataFrame containing routes. Ideally used after clip_routes_to_polygon.

    Returns:
            times : DataFrame
                Start and end times for each unique route.
    """

    # Check if the id column exists in clipped_routes
    assert 'id' in clipped_routes.columns, "No 'id' columns found."

    # Converting ID to id if found
    if 'ID' in clipped_routes.columns:
        print('The column ID was found, but will be converted to lower case.')
        clipped_routes.rename(
            columns = {'ID':'id'},
            inplace = True
            )
    else:
        continue

    # Check if the id column exists in clipped_routes
    assert 'time' in clipped_routes.columns, "No 'time' columns found."

    # Checking time column is a datetime type
    clipped_routes['time'].dtype == '<M8[ns]'

    # Calculating the entry and exit times of each route within the subset of routes - FUNCTION
    timein = clipped_routes.groupby(['id']).first()
    timeout = clipped_routes.groupby(['id']).last()

    # Join the entry and exit times together for each route id
    times = timein.join(timeout, lsuffix='_in', rsuffix='_out')[['time_in', 'time_out']]
    times['duration'] =  times['time_out'] - times['time_in']

    return times

In [ ]:
display(times)